# Data parts for hf transformers

In [1]:
# default_exp hf.data

In [2]:
# export
from forgebox.imports import *
from forgebox.category import Category
from typing import List, Dict, Callable, Any, Tuple

## Process IOBES files

In [ ]:
# export
def convert_iob2_file_to_iobes(file_path, result_path):
    """
    Convert IOB2 file to IOBES
    """
    with open(file_path, 'r') as f:
        lines = f.readlines()
    with open(result_path, 'w') as f:
        for line in lines:
            line = line.strip()
            if line == '':
                f.write('\n')
                continue
            line = line.split()
            if line[-1] == 'O':
                f.write(' '.join(line) + '\n')
            else:
                f.write(' '.join(line[:-1]) + ' ' + line[-1] + '\n')


def conbine_iobes_file(
    file_paths: List[Path],
    new_file_path: Path
):
    """
    Conbine from multiple IOBES files
        into IOBES files
    """
    with open(new_file_path, 'w') as new_file:
        for file_path in file_paths:
            with open(file_path, 'r') as file:
                for line in file:
                    new_file.write(line)

## Dataset

In [ ]:
# export
class IOBES(Dataset):
    """
    Load iobes file for NER training task
    """

    def __init__(
        self,
        file_path,
        tokenizer,
        max_len=128,
        save_buffer: int = 15,
        category: Category = None,
        return_string: bool = False,
        use_frag: bool = False,
    ):
        """
        file_path,
        tokenizer,
        max_len=128,
        save_buffer: int = 15,
        category: Category = None,
            label categories, if set to None, will be figured out
            automatically.
            You can set this to None for train dataset, but for valid
            dataset:
            valid_ds = IOBES(...,category=train_ds.cates)
        return_string: bool = False, do we return original string
            for tokenizer output, this option is good for debuging
            but the data won't pass into cuda if choose so
        use_frag: bool = False, do we use prepend like 'I-','B-'
        """
        self.file_path = file_path
        self.max_len = max_len
        self.pairs = []
        self.list_of_words = []
        self.list_of_labels = []
        self.tokenizer = tokenizer
        self.cates = category
        self.return_string = return_string
        self.use_frag = use_frag
        self.load_data(save_buffer)

    def load_data(self, save_buffer: int = 15):
        """
        Load file in to object structure
        """
        with open(self.file_path, 'r') as f:
            for line in f:
                line = line.strip()
                if line:
                    splited = line.split()
                    if len(splited) != 2:
                        continue
                    word, label = splited
                    # do we use 'I-', 'B-' etc
                    if self.use_frag is False:
                        if "-" in label:
                            label = label.split('-')[1]
                    self.pairs.append([word, label])

        self.pairs = np.array(self.pairs)

        if self.cates is None:
            labels_df = pd.DataFrame({"label": self.pairs[:, 1]})
            self.cates = Category(list(labels_df.vc("label").index))

        self.batching_words(save_buffer)

    def batching_words(self, save_buffer: int = 15):
        """
        batching self.words into self.list_of_words
        by self.max_len -15
        """
        for i in range(0, len(self.pairs), self.max_len-save_buffer):
            chunk_slice = slice(i, i+self.max_len-save_buffer)
            self.list_of_words.append(self.pairs[chunk_slice, 0])
            self.list_of_labels.append(self.pairs[chunk_slice, 1])

    def __len__(self) -> int:
        return len(self.list_of_words)

    def __getitem__(self, idx: int) -> Tuple[List[str]]:
        return list(self.list_of_words[idx]), list(self.list_of_labels[idx])

    def __repr__(self):
        return f"""NER dataset using IOBES annotation
        {len(self)} sentences,
        Labels:
        {list(self.cates.i2c)}
        """

    def collate_fn(self, data):
        """
        data: list of tuple
        """
        words, text_labels = zip(*data)

        inputs = self.tokenizer(
            list(words),
            return_tensors='pt',
            padding=True,
            truncation=True,
            max_length=self.max_len,
            is_split_into_words=True,
            return_offsets_mapping=True,
            add_special_tokens=False,
        )
        return self.align_offsets(inputs, text_labels, words)

    def align_offsets(
        self,
        inputs,
        text_labels: List[List[str]],
        words: List[List[str]]
    ):
        """
        inputs: output if tokenizer
        text_labels: labels in form of list of list of strings
        words: words in form of list of list of strings
        """
        labels = torch.zeros_like(inputs.input_ids).long()
        labels -= 100
        text_lables_array = np.empty(labels.shape, dtype=object)
        words_array = np.empty(labels.shape, dtype=object)
        max_len = inputs.input_ids.shape[1]

        for row_id, input_ids in enumerate(inputs.input_ids):
            word_pos = inputs.word_ids(row_id)
            for idx, pos in enumerate(word_pos):
                if pos is None:
                    continue
                if pos <= max_len:
                    labels[row_id, idx] = self.cates.c2i[text_labels[row_id][pos]]
                    if self.return_string:
                        text_lables_array[row_id,
                                          idx] = text_labels[row_id][pos]
                        words_array[row_id, idx] = words[row_id][pos]

        inputs['labels'] = labels
        if self.return_string:
            inputs['text_labels'] = text_lables_array.tolist()
            inputs['word'] = words_array.tolist()
        return inputs

    def dataloader(self, batch_size: int = 32, shuffle: bool = True):
        """
        Create dataloader
        """
        return DataLoader(
            self,
            batch_size=batch_size,
            shuffle=shuffle,
            collate_fn=self.collate_fn,
        )

    def one_batch(self, batch_size: int = 32, shuffle: bool = True):
        return next(iter(self.dataloader(batch_size, shuffle)))

    def visualize_batch(self, batch, row_idx=0):
        return list(zip(self.tokenizer.convert_ids_to_tokens(batch.input_ids[row_idx]),
                        batch.labels[row_idx].numpy(),
                        batch.text_labels[row_idx],
                        batch.word[row_idx],
                        batch.offset_mapping[row_idx].numpy(),
                        ))

    def set_hfconfig(self, config):
        """
        set the category information to huggingface config
        """
        config.num_labels = len(self.cates)
        config.id2label = {i: label for i, label in enumerate(self.cates.i2c)}
        config.label2id = {label: i for i, label in enumerate(self.cates.i2c)}

In [4]:
from transformers import AutoTokenizer

In [5]:
tokenizer = AutoTokenizer.from_pretrained("raynardj/roberta-pubmed", add_prefix_space=True)

In [6]:
dataset = IOBES("/Users/xiaochen.zhang/data/valid.iobes", tokenizer)

In [7]:
for w,l in zip(*dataset[2]):
    print(f"{w}-{l}")

in-O
blood-O
;-O
content-O
of-O
cAMP-O
was-O
also-O
decreased-O
in-O
lymphocytes-O
by-O
33-O
%-O
.-O
At-O
the-O
same-O
time-O
,-O
total-O
content-O
of-O
T-cell_type
lymphocytes-cell_type
was-O
decreased-O
1.5-fold-O
in-O
peripheric-O
blood-O
.-O
Treatment-O
with-O
I-hydroxyvitamin-O
D3-O
(-O
1-1.5-O
mg-O
daily-O
,-O
within-O
4-O
weeks-O
)-O
led-O
to-O
normalization-O
of-O
total-O
and-O
ionized-O
form-O
of-O
Ca2+-O
and-O
of-O
25-O
(-O
OH-O
)-O
D-O
,-O
but-O
did-O
not-O
affect-O
the-O
PTH-O
content-O
in-O
blood-O
.-O
Concentration-O
of-O
the-O
receptors-protein
to-O
1.25-O
(-O
OH-O
)-O
2D3-O
was-O
elevated-O
up-O
to-O
39.7-O
fmole/mg-O
after-O
I-O
week-O
of-O
the-O
treatment-O
,-O
whereas-O
it-O
was-O
decreased-O
to-O
the-O
initial-O
level-O
24.8-O
fmole/mg-O
within-O
4-O
weeks-O
;-O
simultaneous-O
alteration-O
in-O


In [8]:
dataset.one_batch()

{'input_ids': tensor([[   19,  3741,  2603,  ...,  1417,  2617, 11576],
        [ 4590,  2156,   255,  ...,   405,  1182,  6608],
        [ 6214, 25683,  3809,  ...,    11,     5,  8151],
        ...,
        [13998, 25326,  2413,  ...,     5,  2199,    21],
        [11299,   705, 24811,  ...,   134,  1589,  2032],
        [ 5804,   924,    14,  ...,   366,  1168,     9]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), 'offset_mapping': tensor([[[ 1,  4],
         [ 1,  2],
         [ 2,  5],
         ...,
         [ 3,  5],
         [ 5,  8],
         [ 1,  6]],

        [[ 1,  5],
         [ 1,  1],
         [ 1,  1],
         ...,
         [ 5,  7],
         [ 7,  9],
         [ 9, 14]],

        [[ 1,  5],
         [ 5,  8],
         [ 8, 10],
         ...,
         [ 1,  2],
         [ 1,  3],